<p></p><font size=8 face="courier" color="magenta">Visualize</font>

This notebook creates the widget inside of the "Visualize" section of the [Cytoscape app](../Cytoscape.ipynb).

# Setup Notebook

In [1]:
""" DEBUG  
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug
%run "Source Code/Setup Source.ipynb"   #<-- Debug
# %cd sets the path to that of Cytoscape.ipynb
"""

/root/Documents


The **<font color="magenta">create</font>** object holds all the code for this notebook

# <font color="magenta">Visualizer</font>

The <font color="magenta">Visualizer</font> Class is what is used to create and manage the networks visualization

In [95]:
class Graph():
    def __init__(self):
        #self.w = type("widgets",(),{})() #<-- this is where all the widgets go
        pass

#  <font color="blue">Functions</font>

##  <font color="blue">canvasHtml</font>

<font color="blue">canvasHtml</font> creates the html that the network will be displayed in

In [96]:
def canvasHtml(self): 
    display(HTML("""
    <style> .cy { width: 50vw; height: 30vw}</style>
    <div id="cy" class="cy"></div>
    """))
Graph.canvasHtml = canvasHtml

## <font color="blue">start</font>

<font color="blue">start</font> makes it possible to edit the canvas that displays the network

In [97]:
def start(self):
    JS("""
    var cyto;

    require(['cytoscape'], function(cytoscape){

        cyto = cytoscape({ container: document.getElementById('cy'), 
                layout: { name: 'cose' }, 
                style: [ { selector: 'node', style: { 'height': 20, 'width': 20, 'background-color': 'gray' } },
                 { selector: 'edge', style: { 'width': 5, 'opacity': 0.5, 'line-color': 'gray'} } ]
            });
    });
    """)
Graph.start = start

## <font color="blue">addNode</font>

<font color="blue">addNode</font> adds a nodes to the network

**<font color="red">Function NOT Implemented</font>**

In [98]:
def addNode(b):
    w_id= Graph.w.addNode.id.value
    JS("""
    cyto.add([
      { group: "nodes", data: { id: '{w_id}' } , position: { x: 100, y: 200 } },
    ]);
    """)
    #display( self.w.addNode.final )
    #display(HTML("<h1>HOWDY!</h1>"))
Graph.addNode = addNode

## <font color="blue">addEdge</font>

<font color="blue">addEdge</font> adds an edge to the network

**<font color="red">Function NOT Implemented</font>**

In [75]:
def addEdge(self, source, target, id):
    JS("""
    cyto.add([
      { group: "edges", data: { id: "e0", source: "n0", target: "n1" } }
    ]);
    """)
Graph.addEdge = addEdge

#  <font color="green">Widgets</font>

In [99]:
Graph.w = type("widgets",(),{})()

## <font color="green">Add Node</font>

In [100]:
Graph.w.addNode = type("widget",(),{})()

In [101]:
Graph.w.addNode.head = ipw.HTML("""<h4>Describe the New Node, then Click <font color="blue">Add</font></h4>""")
Graph.w.addNode.id = ipw.Text(description='ID :')

Graph.w.addNode.button = ipw.Button(description='Add', button_style='info')
Graph.w.addNode.button.on_click( Graph.addNode )  #<-- function when widget is clicked

Graph.w.addNode.show = ipw.VBox( [Graph.w.addNode.head, Graph.w.addNode.id, Graph.w.addNode.button ] )
Graph.w.addNode.show

In [86]:
cy.w.addNode

# Scratch Paper

In [102]:
cy = Graph()

In [103]:
cy.canvasHtml()

In [104]:
cy.start()

In [105]:
cy.w.addNode.show

In [47]:
cy.addNode()

NameError: name 'god' is not defined

In [20]:
cy.addNode("asd")

TypeError: addNode() takes 1 positional argument but 2 were given

# Canvas Functions

## Turn On Canvas

In [40]:
def start():
    JS("""
    var cyto;

    require(['cytoscape'], function(cytoscape){

        cyto = cytoscape({ container: document.getElementById('cy_test'), 
                layout: { name: 'cose' }, 
                style: [ { selector: 'node', style: { 'height': 20, 'width': 20, 'background-color': 'gray' } },
                 { selector: 'edge', style: { 'width': 5, 'opacity': 0.5, 'line-color': 'gray'} } ]
            });
    });
    """)
visual.start = start

In [11]:
visual

In [8]:
start()

In [16]:
def start():
    JS("""
    var cyto;

    require(['cytoscape'], function(cytoscape){

        cyto = cytoscape({ container: document.getElementById('cy'), 
                layout: { name: 'cose' }, 
                style: [ { selector: 'node', style: { 'height': 20, 'width': 20, 'background-color': 'gray' } },
                 { selector: 'edge', style: { 'width': 5, 'opacity': 0.5, 'line-color': 'gray'} } ]
            });
    });
    """)
visual.start = start

In [18]:

JS("""
cyto.add([
  { group: "nodes", data: { id: "n0" } , position: { x: 100, y: 200 } },
  { group: "nodes", data: { id: "n1" }, position: { x: 200, y: 100 } },
  { group: "edges", data: { id: "e0", source: "n0", target: "n1" } }
]);
""")


In [17]:
visual.start()